In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
from tqdm import tqdm
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, BatchNormalization, Activation, Input
from keras.models import Sequential, Model
from sklearn.metrics import confusion_matrix, classification_report
from keras import regularizers
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

warnings.filterwarnings("ignore")

2023-06-11 23:31:37.328835: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-11 23:31:37.845686: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-11 23:31:37.850812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 23:31:41.462173: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
res = 216

def load_data(directory):
    images = []
    labels = []
    for folder in tqdm(os.listdir(directory)):
        label = folder
        data_path = os.path.join(directory, folder)
        if not os.path.isdir(data_path):
            continue
        for filename in tqdm(os.listdir(data_path), position=0, leave=True, desc=f"Loading images from {directory.split(os.sep)[-2]} folder"):
            img = cv2.imread(os.path.join(data_path, filename))
            if img is None:
                continue
            img = cv2.resize(img, (res, res))
            img = np.array(img)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)


In [3]:
x_train, y_train = load_data('chest_xray/chest_xray/train/')
x_test, y_test = load_data('chest_xray/chest_xray/test/')

100%|██████████| 3/3 [00:21<00:00,  7.15s/it]


In [4]:
y_train = np.where(y_train == 'NORMAL', 0, 1)
y_test = np.where(y_test == 'NORMAL', 0, 1)

In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# define the model structure
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(216, 216, 3), pooling='avg')
model = Sequential()
model.add(base_model)
model.add(Dense(1, activation='sigmoid'))  # For binary classification use 1 neuron with 'sigmoid' activation

# freeze the base model
base_model.trainable = False

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

# you can then optionally fine-tune the entire model
# unfreeze the base model
base_model.trainable = True
# recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# fine-tune the model
history_fine = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)


Epoch 1/5


2023-06-11 23:41:03.655305: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 730073088 exceeds 10% of free system memory.
2023-06-11 23:41:05.676425: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 382205952 exceeds 10% of free system memory.
2023-06-11 23:41:05.887552: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 382205952 exceeds 10% of free system memory.
2023-06-11 23:41:06.538390: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2023-06-11 23:41:06.649587: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 191102976 exceeds 10% of free system memory.


 19/163 [==>...........................] - ETA: 9:47 - loss: 1.1417 - accuracy: 0.6546

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# make predictions on the test set
y_pred = model.predict(x_test)

# since this is a binary classification problem and our model uses a sigmoid function
# we will convert probabilities to class labels using a threshold of 0.5
y_pred = (y_pred > 0.5).astype(int)

# generate and print the classification report
report = classification_report(y_test, y_pred)
print(report)